BI Goals:

i) investigar as ligações entre a atividade do fogo, conforme medido pelo FRP (Fire Radiative Power), e as concentrações de poluentes e avaliar a zona espacial e temporal de influência da atividade dos incêndios florestais.

(ii) investigar a utilização de FRP como ferramenta para filtrar a contribuição do fumo de biomassa para os registos de poluição atmosférica em bacias atmosféricas urbanas, nomeadamente as emissões de carbono resultantes de incêndios florestais graves.

(iii) desenvolver abordagens multirriscos para caracterizar o comportamento conjunto de múltiplos perigos e riscos consequentes e avaliar o papel desempenhado por condições anteriores e simultâneas de seca e/ou calor na exacerbação de incêndios rurais e consequentes ondas de fumo

In [ ]:
# Gaussian Plume Model Implementation with Supporting Code
# ---------------------------------------------------------
# Assumes user has gridded daily FRP (MW) and wind data (ERA5 u10, v10), pollutant concentrations (CAMS), and grid/mask data.

import numpy as np
import xarray as xr
import pandas as pd
import math

# -----------------------------
# 1. Compute Wind Speed and Direction from u and v
# -----------------------------
def compute_wind_speed_direction(u10, v10):
    wind_speed = np.sqrt(u10**2 + v10**2)
    wind_dir = (np.arctan2(-u10, -v10) * 180 / np.pi) % 360
    return wind_speed, wind_dir

# -----------------------------
# 2. Estimate Atmospheric Stability Class
# -----------------------------
def classify_stability(wind_speed, cloud_cover, ssrd, hour):
    if hour >= 6 and hour <= 18:  # daytime
        if ssrd > 700 and wind_speed < 2:
            return 'A'
        elif ssrd > 500:
            return 'B'
        elif ssrd > 250:
            return 'C'
        else:
            return 'D'
    else:  # nighttime
        if cloud_cover > 0.5:
            return 'D'
        elif wind_speed < 2:
            return 'F'
        else:
            return 'E'

# -----------------------------
# 3. Compute Dispersion Coefficients
# -----------------------------
def dispersion_coefficients(x, stability):
    if stability == 'A':
        sigma_y = 0.22 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.20 * x
    elif stability == 'B':
        sigma_y = 0.16 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.12 * x
    elif stability == 'C':
        sigma_y = 0.11 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.08 * x * (1 + 0.0002 * x)**-0.5
    elif stability == 'D':
        sigma_y = 0.08 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.06 * x * (1 + 0.0015 * x)**-0.5
    elif stability == 'E':
        sigma_y = 0.06 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.03 * x * (1 + 0.0003 * x)**-1
    elif stability == 'F':
        sigma_y = 0.04 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.016 * x * (1 + 0.0003 * x)**-1
    else:
        sigma_y = sigma_z = np.nan
    return sigma_y, sigma_z

# -----------------------------
# 4. Estimate Emission Rate Q from FRP (MW) and Emission Factor (EF in g/MJ)
# -----------------------------
def compute_emission_rate(FRP_MW, EF_g_per_MJ):
    # Convert FRP from MW to MJ/s
    FRP_MJ_s = FRP_MW * 1000
    Q_grams_per_s = FRP_MJ_s * EF_g_per_MJ
    Q_kg_per_s = Q_grams_per_s / 1000
    return Q_kg_per_s

# -----------------------------
# 5. Gaussian Plume Model (ground-level, steady-state)
# -----------------------------
def gaussian_plume_ground(Q, u, x, y, H, sigma_y, sigma_z):
    factor = Q / (2 * np.pi * u * sigma_y * sigma_z)
    exp_y = np.exp(- (y**2) / (2 * sigma_y**2))
    exp_z = np.exp(- (H**2) / (2 * sigma_z**2))
    C = factor * exp_y * exp_z
    return C

# -----------------------------
# 6. Apply Regional Mask for Aggregation
# -----------------------------
def apply_region_mask(concentration, mask):
    masked = np.where(mask == 1, concentration, np.nan)
    regional_mean = np.nanmean(masked)
    return regional_mean


Portugal

Spain

Italy

Greece